# Tutorial: Hierarchical Recurrent Connectivity

This notebook shows how to create hierarchical connectivity patterns and use them in network architectures.

---

## 🔊 NOISE CONFIGURATION: ENABLED (Default)

> **This tutorial runs with NOISE INJECTION (documented defaults).**
>
> | Parameter | Default | Description |
> |-----------|---------|-------------|
> | `phi` | **0.01** | Noise on input flux |
> | `s` | **0.005** | Noise on state |
> | `relative` | **False** | Absolute scaling |
>
> **To toggle noise on/off:** Use the `NOISE_ENABLED` variable in the next code cell.

---

**Key concepts:**
- Building multi-tier connectivity patterns with nested blocks using `structure.hierarchical_blocks()`
- Customizing connectivity density at each tier with `tier_fractions`
- Visualizing hierarchical network structure with automatic block detection
- Understanding hierarchical mask properties (sparsity, fan-out, etc.)

In [ ]:
# Setup: Ensure soen_toolkit is importable
import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np

from soen_toolkit.core.utils.hierarchical_mask import analyze_mask, create_hierarchical_mask
from soen_toolkit.nn import Graph, init, layers, structure

In [ ]:
# ==============================================================================
# NOISE CONFIGURATION TOGGLE
# ==============================================================================
# Set NOISE_ENABLED = False to run with ideal conditions (no noise)
# Set NOISE_ENABLED = True for noise injection (default)

NOISE_ENABLED = True  # Toggle this to enable/disable noise

# Default noise parameters (documented defaults)
NOISE_DEFAULTS = {
    "phi": 0.01,           # Noise on input flux
    "s": 0.005,            # Noise on state
    "g": 0.0,              # Source function noise
    "bias_current": 0.0,   # Bias current noise
    "j": 0.0,              # Connection weight noise
    "relative": False,     # Absolute scaling
}

def set_model_noise(model, enabled=True, noise_values=None):
    """
    Toggle noise injection on/off for a SOEN model.
    
    Args:
        model: SOENModelCore instance
        enabled: If True, apply noise; if False, set all noise to 0
        noise_values: Dict of noise parameters (uses NOISE_DEFAULTS if None)
    
    Returns:
        model: The modified model (for chaining)
    """
    from soen_toolkit.core.configs import NoiseConfig
    
    if noise_values is None:
        noise_values = NOISE_DEFAULTS
    
    # Update layer noise configurations
    for cfg in model.layers_config:
        if enabled:
            cfg.noise = NoiseConfig(
                phi=noise_values.get("phi", 0.01),
                s=noise_values.get("s", 0.005),
                g=noise_values.get("g", 0.0),
                bias_current=noise_values.get("bias_current", 0.0),
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras=getattr(cfg.noise, "extras", {}),
            )
        else:
            cfg.noise = NoiseConfig(
                phi=0.0, s=0.0, g=0.0, bias_current=0.0, j=0.0,
                relative=False,
                extras=getattr(cfg.noise, "extras", {}),
            )
    
    # Update connection noise configurations
    for conn_cfg in model.connections_config:
        if enabled:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0,
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras={},
            )
        else:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0, j=0.0,
                relative=False, extras={},
            )
    
    status = "ENABLED" if enabled else "DISABLED"
    print(f"✓ Noise injection {status}")
    if enabled:
        print(f"  phi={noise_values['phi']}, s={noise_values['s']}, "
              f"relative={noise_values['relative']}")
    
    return model

print(f"Noise injection: {'ENABLED' if NOISE_ENABLED else 'DISABLED'}")
if NOISE_ENABLED:
    print(f"  Default values: phi={NOISE_DEFAULTS['phi']}, s={NOISE_DEFAULTS['s']}, "
          f"relative={NOISE_DEFAULTS['relative']}")

## Create Hierarchical Mask

Creates a 3-tier hierarchy with base_size=4:
- Tier 0: blocks of 4 nodes (100% connected)
- Tier 1: blocks of 16 nodes (10% selected, fully connected)
- Tier 2: blocks of 64 nodes (10% selected, fully connected)

Total: 64 nodes with symmetric connections.


In [ ]:
# Create mask (uses default tier_fractions=[1.0, 0.5, 0.25])
mask = create_hierarchical_mask(levels=3, base_size=4, tier_fractions=[1.0, 0.1, 0.1])
stats = analyze_mask(mask)


# Visualize mask
fig, ax = plt.subplots(figsize=(7, 6))
ax.imshow(mask, cmap="binary", aspect="auto")
ax.set_title(f"Hierarchical Mask ({mask.shape[0]}×{mask.shape[1]})")
ax.set_xlabel("Source")
ax.set_ylabel("Destination")
plt.tight_layout()
plt.show()

In [ ]:
## Build a Model with Custom Hierarchical Masks

# Create two different hierarchical masks
mask1 = create_hierarchical_mask(levels=3, base_size=4)  # 64 nodes
mask2 = create_hierarchical_mask(levels=2, base_size=4)  # 16 nodes

# Use a real temporary directory for mask files
with tempfile.TemporaryDirectory() as temp_dir:
    mask1_file = os.path.join(temp_dir, "mask1.npz")
    mask2_file = os.path.join(temp_dir, "mask2.npz")
    np.savez(mask1_file, mask=mask1)
    np.savez(mask2_file, mask=mask2)

    # Build model with different hierarchies per layer
    g_mixed = Graph(dt=37, network_evaluation_method="layerwise")

    g_mixed.add_layer(0, layers.MultiplierWICC(dim=64))  # WICC (With Collection Coil) multiplier
    g_mixed.connect(
        0,
        0,
        structure=structure.custom(
            mask1_file,
            # Store metadata for visualization
            visualization_metadata={"hierarchical": {"levels": 3, "base_size": 4}},
        ),
        init=init.constant(0.1),
    )

    g_mixed.add_layer(1, layers.MultiplierWICC(dim=16))  # WICC multiplier
    g_mixed.connect(1, 1, structure=structure.custom(mask2_file, visualization_metadata={"hierarchical": {"levels": 2, "base_size": 4}}), init=init.constant(0.1))

    # Connect layers with sparse connectivity
    g_mixed.connect(0, 1, structure=structure.sparse(sparsity=0.1), init=init.xavier_uniform())

    kpis = g_mixed.compute_summary()["kpis"]

    # Visualize the hierarchical structure
    # Each layer uses its stored metadata to show hierarchical blocks
    g_mixed.visualize_grid_of_grids()